In [ ]:
# install packages for colab
!pip install optuna
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x

In [ ]:
# auto reload modules
%load_ext autoreload
%autoreload 0

In [ ]:
# colab specific, remove if locally
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# edit paths
path_to_data = '/content/drive/MyDrive/civol/iceccme/'
filepath_data = path_to_data + 'sample_labeled_by_qwen2_10_runs_FINAL.csv'
modelpath = path_to_data + 'models_qwen/'

In [ ]:
# helper function for bringing data into required structure
def get_dataset_from_df(col_text_a, col_text_b, label_col, df):
    dataset = []
    for index, row in df.iterrows():
        text_a = row[col_text_a]
        text_b = row[col_text_b]
        label = row[label_col]
        dataset.append((text_a, text_b, label))
        dataset.append((text_b, text_a, label))
    return dataset


In [ ]:
# import modules
import logging
from tqdm.autonotebook import tqdm, trange
from sentence_transformers import CrossEncoder, InputExample
from torch.utils.data import DataLoader
from sentence_transformers.cross_encoder.evaluation import CEBinaryClassificationEvaluator
from sklearn.model_selection import train_test_split
import optuna
import pandas as pd
import torch

tqdm

<ipython-input-5-8dfc5c26fce2>:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


tqdm.notebook.tqdm_notebook

In [ ]:
# load data
df_base = pd.read_csv(filepath_data, sep=';')
df_base['title_description'] = df_base['title'] + ': ' + df_base['description']

In [ ]:
# set logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [ ]:
# get data in structure
dataset = get_dataset_from_df('goal', 'title_description', 'final_label', df_base)  #TODO

In [ ]:
# callback for logging loss
current_loss = None


def simple_callback(loss, epoch, steps):
    global current_loss
    current_loss = loss
    logging.log(logging.INFO, f"Loss: {loss}, epoch: {epoch}, steps: {steps}")

In [ ]:
# check for gpu
device = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
def objective(trial):
    try:
        # Log the start of a trial with the trial number
        logging.log(logging.INFO, f"Trial {trial.number}")
        # Log that model creation is starting
        logging.log(logging.INFO, f"Model created")
        # Initialize a CrossEncoder model with predefined settings
        model = CrossEncoder('cross-encoder/stsb-roberta-base', num_labels=1, device=device)
        # Suggest a learning rate using a logarithmic scale within a specified range
        learning_rate = trial.suggest_float("learning_rate", 5e-5, 1e-1, log=True)
        # Suggest the batch size within a range from 2 to 16
        batch_size = trial.suggest_int("batch_size", 2, 16)
        # Suggest the number of epochs for training within a range from 1 to 20
        num_epochs = trial.suggest_int("num_epochs", 1, 20)
        # Suggest the number of warmup steps within a range from 0 to 1000
        warmup_steps = trial.suggest_int("warmup_steps", 0, 1000)
        # Suggest weight decay using a logarithmic scale within a specified range
        weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-1, log=True)
        # Suggest dropout rate within a specified range to prevent overfitting
        dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)

        # Set the dropout rate for the model
        model.dropout_rate = dropout_rate
        # Log all set parameters for debugging purposes
        logging.log(logging.INFO,
                    f"Learning rate: {learning_rate}, batch size: {batch_size}, num epochs: {num_epochs}, warmup steps: {warmup_steps}, weight decay: {weight_decay}, dropout rate: {dropout_rate}")
        # Split the dataset into training and validation sets with a 10% validation split
        train_data, val_data = train_test_split(dataset, test_size=0.1, random_state=347657)

        # Prepare DataLoader for training data
        train_dataloader = DataLoader([InputExample(texts=[t[0], t[1]], label=t[2]) for t in train_data], shuffle=True,
                                      batch_size=batch_size)
        # Prepare DataLoader for validation data
        valid_dataloader = DataLoader([InputExample(texts=[t[0], t[1]], label=t[2]) for t in val_data],
                                      batch_size=batch_size)

        # Prepare validation data for the evaluator
        val_sentence_pairs = [[t[0], t[1]] for t in val_data]
        labels = [t[2] for t in val_data]

        # Initialize the evaluator with the validation data and configuration
        evaluator = CEBinaryClassificationEvaluator(sentence_pairs=val_sentence_pairs, labels=labels, name='goal_eval',
                                                    write_csv=True, show_progress_bar=True)

        # Fit the model with training data, training configurations, and evaluation settings
        model.fit(train_dataloader=train_dataloader,
                  epochs=num_epochs,
                  warmup_steps=warmup_steps,
                  output_path=modelpath + 'model_trial_2_' + str(trial.number),
                  optimizer_params={'lr': learning_rate, 'weight_decay': weight_decay},
                  show_progress_bar=True,
                  evaluator=evaluator,
                  evaluation_steps=1000,
                  save_best_model=True,
                  callback=simple_callback
                  )
        # Log that the model fitting is complete
        logging.log(logging.INFO, f"Model fitted")
        # Evaluate the model using the evaluator and log the accuracy
        accuracy = evaluator(model, output_path=modelpath + 'model_trial_2_' + str(trial.number))
        logging.log(logging.INFO, f"Accuracy: {accuracy}")
        # Report the loss to Optuna and handle pruning if necessary
        global current_loss
        if current_loss:
            trial.report(current_loss, trial.number)
            logging.log(logging.INFO, f"Trial pruned")
            if trial.should_prune():
                raise optuna.TrialPruned()
    except Exception as e:
        # Log any exceptions that occur during the trial
        logging.log(logging.ERROR, f"Error: {e}")
        raise e
    finally:
        # Ensure the model is deleted and GPU memory is cleared after each trial
        if model:
            del model
            if device == "cuda":
                torch.cuda.empty_cache()
            elif device == "mps":
                torch.mps.empty_cache()

    # Return the accuracy if available
    if accuracy:
        return accuracy


TODO: Edit sql Lite Path to drive


In [ ]:
storage_path = 'sqlite:///' + modelpath + 'db_max_qwen.sqlite3'

In [ ]:
study = optuna.create_study(storage=storage_path, study_name="qwen_data_hpo_max_dc_d", load_if_exists=True, direction="maximize")
study.optimize(objective, n_trials=12)

[I 2024-06-27 07:34:48,326] A new study created in RDB with name: qwen_data_hpo_max_dc_d
2024-06-27 07:34:48,376 - INFO - Trial 0
2024-06-27 07:34:48,378 - INFO - Model created
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

2024-06-27 11:05:35,047 - INFO - Accuracy:           70.25	(Threshold: 0.6360)
2024-06-27 11:05:35,048 - INFO - F1:                 82.35	(Threshold: 0.6360)
2024-06-27 11:05:35,050 - INFO - Precision:          70.59
2024-06-27 11:05:35,053 - INFO - Recall:             98.82
2024-06-27 11:05:35,055 - INFO - Average Precision:  69.34

2024-06-27 11:05:35,061 - INFO - Accuracy: 0.693353705118411
2024-06-27 11:05:35,082 - INFO - Trial pruned
[I 2024-06-27 11:05:35,275] Trial 9 finished with value: 0.693353705118411 and parameters: {'learning_rate': 0.000926077844966131, 'batch_size': 11, 'num_epochs': 17, 'warmup_steps': 733, 'weight_decay': 4.525471473182416e-05, 'dropout_rate': 0.11641311745480959}. Best is trial 5 with value: 0.8947331337996255.
2024-06-27 11:05:35,309 - INFO - Trial 10
2024-06-27 11:05:35,310 - INFO - Model created
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in versio

Epoch:   0%|          | 0/6 [00:00<?, ?it/s]

Iteration:   0%|          | 0/69 [00:00<?, ?it/s]

2024-06-27 11:05:38,422 - ERROR - Error: CUDA out of memory. Tried to allocate 192.00 MiB. GPU 
[W 2024-06-27 11:05:38,655] Trial 10 failed with parameters: {'learning_rate': 5.268528030008424e-05, 'batch_size': 16, 'num_epochs': 6, 'warmup_steps': 995, 'weight_decay': 0.06422146703104012, 'dropout_rate': 0.20614980955005321} because of the following error: OutOfMemoryError('CUDA out of memory. Tried to allocate 192.00 MiB. GPU ').
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-13-f35afe2df549>", line 49, in objective
    raise e
  File "<ipython-input-13-f35afe2df549>", line 27, in objective
    model.fit(train_dataloader=train_dataloader,
  File "/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py", line 278, in fit
    model_predictions = self.model(**features, return_dict=True)
  File "/usr/local/lib

OutOfMemoryError: CUDA out of memory. Tried to allocate 192.00 MiB. GPU 

In [ ]:
print(f"Best value: {study.best_value} (params: {study.best_params})")